# How do we stop the Churn Burn?
# Goal: 
* Discover driving features affecting churn
* Use drivers to develop a machine learning model to predict churn
* Use these predictions to inform preemptive decisions aimed at alleviating future churn

# Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

# Decision Tree and Model Evaluation Imports
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text

# import sklearn.metrics
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import plot_confusion_matrix 
from sklearn.metrics import classification_report 

# import environments and helper files
import wrangle as w
import QMCBT_explore_evaluate as ee
#import QMCBT_modeling as m

# set constants
α = 0.05
alpha = 0.05
np.random.seed(1992)
#random_state=1992

#import warnings
#warnings.filterwarnings("ignore")

# Acquire
* telco_churn data from Codeup SQL database was used for this project.
* The data was initially pulled on 26-OCT.
* The initial DataFrame contained 7043 records with 44 features  
    (44 columns and 7043 rows) before cleaning & preparation.
* Each row represents a customer record both current & historical.
* Each column represents a feature provided by telco or an informational element about the customer.

In [2]:
# Use function from wrangle file to acquire telco_churn data
df = w.get_telco_churn_df()

# Prepare
* List steps taken to clean your data here
* In particular call out how you handle null values and outliers in detail
* You must do this even if you do not do anything or do not encounter any
* Any time there is potential to make changes to the data you must be upfront about the changes you make or do not make

**Prepare Actions:**
* **DROP:** Removed 4 index_id, 18 duplicate, and 1 corrupted data column
* **RENAME:** Initially did not need to Rename any original columns
* **NaN/Null:** Only one column contained NaN/nulls in the data (it was in the corrupted field that was removed).
* **REFORMAT:** 2 columns contained inappropriate data types that needed to be reformatted 
* **REPLACE:** 7 columns had a third value that could be determined by another feature, replaced third value in each column with appropriate yes/no value. 1 column had empty non-null values that were replaced with 0
* **ENCODED:** 14 categorical columns from variables to boolean numeric values
* **MELT:** No melts needed
* **PIVOT:** 3 columns with more than two variables were pivotted
* **FEATURE ENGINEER:** No new features were added
* **IMPUTE:** No data was imputed
* **DROP2:** 16 Columns duplicated by Encoded and Pivot Columns were dropped
* **RENAME2:** 13 encoded columns were renamed after original columns were dropped
* **OUTLIERS:** have not been removed for this iteration of the project

In [3]:
# Use function from wrangle file to acquire telco_churn data
df = w.clean_telco(df)

# Split
* **SPLIT:** train, validate and test (approx. 60/20/20), stratifying on target of 'churn'
* **SCALED:** no scaling was conducted


In [4]:
# use predefined function to split data for train, validate, and test
target = 'churn'
train_df, validate_df, test_df = w.split(df, target)

_______________________________________________________________
|                              DF                             |
|-------------------:-------------------:---------------------|
|       Train       |       Validate    |          Test       |
|-------------------:-------------------:---------------------|
| x_train | y_train |   x_val  |  y_val |   x_test  |  y_test |
:-------------------------------------------------------------:

* 1. tree_1 = DecisionTreeClassifier(max_depth = 5)
* 2. tree_1.fit(x_train, y_train)
* 3. predictions = tree_1.predict(x_train)
* 4. pd.crosstab(y_train, y_preds)
* 5. val_predictions = tree_1.predict(x_val)
* 6. pd.crosstab(y_val, y_preds)

Prepared df: (7043, 25)

Train (train_df): (4225, 25)
Validate (validate_df): (1409, 25)
Test (test_df): (1409, 25)


# Data Dictionary
<div class="alert alert-success">

| Feature | Definition |
|:--------|:-----------|
|1|value equals 'Yes' or True|
|0|value equals 'No' or False|    
|senior_citizen|DEFINITION|
|tenure|DEFINITION|
|monthly_charges|DEFINITION|
|total_charges|DEFINITION|
|signup_date|DateTime Format is yyyy-mm-dd HH:MM:SS (example: 2021-04-21 18:07:34)|
|gender|0=Female, 1=Male|
|partner|DEFINITION|
|dependents|DEFINITION|
|phone_service|DEFINITION|
|paperless_billing|DEFINITION|
|churn|DEFINITION|
|multiple_lines|DEFINITION|
|online_security|DEFINITION|
|online_backup|DEFINITION|
|device_protection|DEFINITION|
|tech_support|DEFINITION|
|streaming_tv|DEFINITION|
|streaming_movies|DEFINITION|
|internet_service_type_DSL|This column was dropped for ease of machine learning. If the other two service types are False then this feature is True by default|
|internet_service_type_Fiber optic|DEFINITION|
|internet_service_type_None|DEFINITION|
|contract_type_Month-to-month|DEFINITION|
|contract_type_One year|DEFINITION|
|contract_type_Two year|DEFINITION|
|payment_type_Bank transfer(automatic)|DEFINITION|
|payment_type_Credit card (automatic)|DEFINITION|
|payment_type_Electronic check|DEFINITION|
|payment_type_Mailed check'|DEFINITION|</div>

## A sneak peek at the data

In [5]:
# Print the clean DataFrame transposed for visability
df.head().T

,0,1,2,3,4
senior_citizen,0,0,0,1,1
tenure,9,9,4,13,3
monthly_charges,65.6,59.9,73.9,98.0,83.9
total_charges,593.3,542.4,280.85,1237.85,267.4
signup_date,2021-04-21 18:07:34,2021-04-21 18:07:34,2021-09-21 18:07:34,2020-12-21 18:07:34,2021-10-21 18:07:34
gender,0,1,1,1,0
partner,1,0,0,1,1
dependents,1,0,0,0,0
phone_service,1,1,1,1,1
paperless_billing,1,0,1,1,1


## A Summary of the data

In [7]:
# Describe each feature with basic statistical data
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
senior_citizen,4225.0,0.154320,0.361298,0.00,0.0,0.00,0.00,1.00
tenure,4225.0,32.267456,24.608717,0.00,9.0,28.00,55.00,72.00
monthly_charges,4225.0,64.248414,30.216742,18.25,34.2,70.15,89.75,118.65
total_charges,4225.0,2259.222071,2260.439423,0.00,389.6,1381.20,3762.00,8564.75
gender,4225.0,0.512189,0.499911,0.00,0.0,1.00,1.00,1.00
partner,4225.0,0.493491,0.500017,0.00,0.0,0.00,1.00,1.00
dependents,4225.0,0.308639,0.461986,0.00,0.0,0.00,1.00,1.00
phone_service,4225.0,0.908166,0.288826,0.00,1.0,1.00,1.00,1.00
paperless_billing,4225.0,0.592189,0.491486,0.00,0.0,1.00,1.00,1.00
churn,4225.0,0.265325,0.441559,0.00,0.0,0.00,1.00,1.00


In [12]:
# Use predefined function to count all unique values for each feature.
ee.nunique_column_all(train_df)

0    3573
1     652
Name: senior_citizen, dtype: int64

1     357
72    230
2     139
3     130
71     98
     ... 
38     32
44     30
36     30
30     30
0       8
Name: tenure, Length: 73, dtype: int64

20.05    35
19.85    32
20.25    29
19.90    29
19.70    28
         ..
26.80     1
65.10     1
53.90     1
58.70     1
66.30     1
Name: monthly_charges, Length: 1368, dtype: int64

0.00       8
19.65      6
19.90      5
20.20      5
19.55      5
          ..
150.35     1
4916.40    1
305.95     1
1436.95    1
964.35     1
Name: total_charges, Length: 4005, dtype: int64

2021-12-21 18:07:34    357
2016-01-21 18:07:34    230
2021-11-21 18:07:34    139
2021-10-21 18:07:34    130
2016-02-21 18:07:34     98
                      ... 
2018-11-21 18:07:34     32
2018-05-21 18:07:34     30
2019-01-21 18:07:34     30
2019-07-21 18:07:34     30
2022-01-21 18:07:34      8
Name: signup_date, Length: 73, dtype: int64

1    2164
0    2061
Name: gender, dtype: int64

0    2140
1    2085
Name: par

# Explore
Here you will explore your data then highlight 4 questions that you asked of the data and how those questions influenced your analysis
Remember to split your data before exploring how different variables relate to one another
Each question should be stated directly
Each question should be supported by a visualization
Each question should be answered in natural language
Two questions must be supported by a statistical test, but you may choose to support more than two
See the following example, and read the comments in the next cell

## 1. Is there a relationship between Tenure and Churn
* ```Tenure``` has a significant ```relationship``` with ```Churn```.

## 1.1 Is the average Tenure of Active customers greater than the average Tenure of Churned customers?
* The ```average Tenure``` of ```Active``` customers is ```less than or equal``` to the ```average Tenure``` of customers that ```Churn```.

## 2. Is there a relationship between monthly_charges and Churn?
* ```Monthly Charges``` has a significant ```Relationship``` with ```Churn```.

## 2.1 Are the average monthly charges of customers that Churn higher than the average monthly charges of Active customers?
* ```Average monthly charges``` of customers that ```Churn``` are ```less than or equal``` to the ```average monthly charges``` of ```Active``` customers.

## 3. Is there a difference between tech support and Churn?
* ```Tech Support``` has a significant ```Relationhip``` with ```Churn```

## 3.1 Is the average of customer Churn without Tech Support greater than the average of customer Churn with Tech Support?
* The ```average Churn``` of customers ```without Tech Support``` is ```greater than``` the ```average Churn``` of customers ```with Tech Support```

## 3.2 Is the average of customer Churn without Tech Support greater than the average of Active customers without tech support?
* The ```average``` of customer ```Churn without Tech Support``` is ```greater than``` the ```average of Active customers with Tech Support```

## OVERALL TAKEAWAYS OF CHURN AND TECH SUPPORT

<div class="alert alert-info">
<b>30% of all customers without tech support churn</b>
</div>

<div class="alert alert-info">
<b>82% of all churn is attributed to not having tech support</b>
</div>

<div class="alert alert-info">
<b>Only 17% of customers with tech support churn</b>
</div>

<div class="alert alert-info">
<b>Only 18% of all churn can be attributed to customers with tech support</b>
</div>